In [6]:
%matplotlib inline

from matplotlib import pyplot as plt

import numpy as np
import pycuda
import pycuda.compiler as cuda_compiler
import pycuda.driver as cuda_driver

from Timer import Timer
import IPythonMagic

import pytest
from ipytest import clean_tests, run_pytest

In [7]:
#These were from the IPythonMagic module
%setup_logging
%cuda_context_handler context

Global logger already initialized!
Registering context in user workspace
Context already registered! Ignoring


In [22]:
#At first solve the problem with the CPU
class HeatEquation1D(object):
    """
    This class approximates the solution of the heat equation
    using an explicit finite difference scheme
    """
    #now build the constructor
    def __init__(self, u0, kappa, dx, dt):
        #now assign these variables to the self
        self.u0 = u0.copy()
        self.u1 = np.empty_like(self.u0)
        self.kappa = kappa
        self.dx = dx
        self.dt = dt
        #these are a float apart from u0, numpy array. we have now 2 pointers pointing at the same memory, so let's do a copy
        
        #nx is u0.shape minus two because of boundaries
        self.nx = self.u0.shape[0] - 2
        
    def step(self):
        for i in range(1, self.nx + 1):
            self.u1[i] = self.u0[i] \
                + self.kappa*self.dt/self.dx**2 \
                * (self.u0[i-1] - 2.0*self.u0[i] + self.u0[i+1])
            
        self.u1[0] = self.u1[1]
        self.u1[self.nx+1] = self.u1[self.nx]
            
    def download(self):
        #called download because we'll move to GPU
        return self.u1.copy()
            

In [24]:
u0 = np.zeros(7)
u0[3] = 1.0
kappa = 1.0
dx = 1.0
dt = 0.4*dx**2.0/(2.0*kappa)

simulator = HeatEquation1D(u0, kappa, dx, dt)
simulator.step()
result = simulator.download()
print(result)

[0.  0.  0.2 0.6 0.2 0.  0. ]


In [11]:
clean_tests()

def test_HeatEquation1D():
    u0 = np.zeros(15)
    kappa = 1.0
    dx = 1.0
    dt = 0.4*dx**2.0/(2.0*kappa)

    simulator = HeatEquation1D(u0, kappa, dx, dt)

    assert np.all(u0 == simulator.u0)
    assert simulator.u0.shape == simulator.u1.shape
    assert kappa == simulator.kappa
    assert dx == simulator.dx
    assert dt == simulator.dt
    
run_pytest(filename='HeatEquation1DFabio.ipynb', pytest_options=['-vvv'])

============================= test session starts ==============================
platform linux -- Python 3.6.6, pytest-3.8.2, py-1.6.0, pluggy-0.7.1 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /home/ubuntu/jupyter_notebooks/Fabio/MilanoGPU2018/notebooks, inifile:
collecting ... collected 1 item

HeatEquation1DFabio.py::test_HeatEquation1D <- <ipython-input-11-c15c67504d3e> PASSED [100%]

=========================== 1 passed in 0.02 seconds ===========================


0